In [5]:
import os
import sys
import subprocess
import filecmp
import time
import difflib
import re

compiler_path = "./build/compiler"
test_folder="./functional2025"
pass_args=["--test=mem2reg,DCE,sccp"]

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

CE_list = []
LLI_FAIL_list = []
WA_list = []
AC_list = []
TLE_list = []
Time_Out = []
total_time=0
Bad_test = []

for test in test_list:
    if test.endswith(".sy"):
        # add test_args to list
        print("Run pass: "+test)
        compile_args=[compiler_path, test]
        for arg in pass_args:
            compile_args.append(arg)
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            CE_list.append(test)
            print("Compiler Error:"+test)
            continue
        # run lli
        ll_file = test+".ll"
        out_file = test[:-2]+"out"
        # riscv file
        # asm_file = test + "..s"
        # maybe theres no input file
        
        if not os.path.exists(test[:-2]+"in") and not os.path.exists(out_file):
            Bad_test.append(test)
            continue
        
        # Kill program run over 5s, give it a TLE
        try:
            if not os.path.exists(test[:-2]+"in"):
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
            else:
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
        except subprocess.TimeoutExpired:
            print("Timeout Error")
            TLE_list.append(test)
            print(test)
            total_time+=15
            continue
        #timer set
        try:
            if not os.path.exists(test[:-2] + "in"):
                ret = subprocess.run(["time", "lli-14", "-opaque-pointers", ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
            else:
                with open(test[:-2] + "in") as input_file:
                    input_data = input_file.read().encode()
                    ret = subprocess.run(["time", "lli-14", "-opaque-pointers", ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=input_data, timeout=10)
        
            stderr_output = ret.stderr.decode()    
            match = re.search(r'TOTAL: (\d+)H-(\d+)M-(\d+)S-(\d+)us', stderr_output)
            if match:
                hours, minutes, seconds, microseconds = map(int, match.groups())
                # 将时间转换为秒
                total_seconds = hours * 3600 + minutes * 60 + seconds + microseconds / 1e6
                # 假设total_time已经定义，并且是以秒为单位的
                total_time += total_seconds
            else:
                print("No TOTAL time found in stderr_output.")
        except subprocess.TimeoutExpired:
            continue
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str=ret.stdout.decode()
        if dump_str and not dump_str.endswith('\n'):
            dump_str += "\n"
        dump_str += str(ret.returncode) + "\n"
        std_output=open(out_file).read()
        diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
        if(len(list(diff))!=0):
            print("Wrong Answer")
            WA_list.append(test)
            print(test)
            continue
        # print("Accepted")
        AC_list.append(test)
    
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Runtime Error: Total: "+str(len(LLI_FAIL_list)))
print("Timeout Error: Total: "+str(len(TLE_list)))
print("Wrong Answer: Total: "+str(len(WA_list)))
print("TimeOut Function:"+str(len(Time_Out)))
print("Bad Test: Total: "+str(len(Bad_test)))
print("Accepted: Total: "+str(len(AC_list)))
print("TOTAL TIME:"+str(total_time))

Run pass: ./functional2025/h_functional/05_param_name.sy
Run pass: ./functional2025/h_functional/39_fp_params.sy
Run pass: ./functional2025/h_functional/24_array_only.sy
Run pass: ./functional2025/h_functional/06_func_name.sy
Run pass: ./functional2025/h_functional/08_global_arr_init.sy
Run pass: ./functional2025/h_functional/36_rotate.sy
Run pass: ./functional2025/h_functional/22_matrix_multiply.sy
Run pass: ./functional2025/h_functional/25_scope3.sy
Run pass: ./functional2025/h_functional/27_scope5.sy
Run pass: ./functional2025/h_functional/00_comment2.sy
Run pass: ./functional2025/h_functional/04_break_continue.sy
Run pass: ./functional2025/h_functional/01_multiple_returns.sy
Run pass: ./functional2025/h_functional/30_many_dimensions.sy
Run pass: ./functional2025/h_functional/17_maximal_clique.sy
Run pass: ./functional2025/h_functional/31_many_indirections.sy
Run pass: ./functional2025/h_functional/11_BST.sy
Run pass: ./functional2025/h_functional/15_graph_coloring.sy
Run pass: ./fu